In [8]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from small2DNet import small2DNet
from small3DNet import small3DNet
from colorMNist import colorMNist
import torch
import pickle

In [9]:
def avg_acc(x, y, mtype, dataset_length, n_exp, model_layers2, model_layers3, features, hue_shift=-1):
    accuracies = []
    for iteration in tqdm(range(2, 13, 2), total=6, desc=f"{x} {y} {mtype} {dataset_length}"):
        # Load dataset
        with open("custom_datasets/" + str(dataset_length) + '/' + y + ".pkl", "rb") as f:
            cmnist_train, cmnist_val, cmnist_test = pickle.load(f)
            test_dataset = colorMNist(cmnist_test)
            test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers = 0)

            # Layers of the model
            model_layers = model_layers2 if mtype == 2 else model_layers3
            # Create model
            model = small2DNet(model_layers, model_layers[-2], features[0]) if mtype == 2 else small3DNet(model_layers, model_layers[-2], features[1])
            # Load file
            extra = "" if hue_shift == -1 else "_" + str(hue_shift)
            lfile = "C-Gaussian" + str(mtype) + "D_" + x + extra
            # Load model
            model.load_state_dict(torch.load('Experiments/cmnist/model_saves/' + str(n_exp) + '/' + str(dataset_length)
                                             + '/' + str(iteration) + '/' + lfile + '.pth'))
            # model.load_state_dict(torch.load('testsave/' + lfile + '.pth'))
            # Put model on gpu
            model.cuda()

            # Total and amount correct
            test_correct = 0
            test_total = 0

            # Put the model in evaluation mode
            model.eval()

            # Without gradient calculation
            with torch.no_grad():
                for (images, labels) in test_dataloader:

                    # Add extra dimension for the network
                    if mtype == 3:
                        images = images.unsqueeze(1)

                    # Put images
                    images = images.cuda()

                    # Predicted labels
                    preds = model(images)

                    # Top predictions per image
                    _, top_preds = torch.max(preds, 1)

                    # Predictions and images back on cpu
                    top_preds = top_preds.cpu()
                    images = images.cpu()

                    # Amount of correct predictions
                    predictions = [top_preds[i].item() == labels[i].item() for i in range(len(labels))]
                    correct = np.sum(predictions)

                    # Update total correct and total images
                    test_correct += correct
                    test_total += len(images)

            accuracies.append(test_correct / test_total)
    return accuracies

In [ ]:
datasets = [["0", "cmnist_deterministic"], ["12", "cmnist_gaussian_12"], ["60", "cmnist_gaussian_60"],
            ["120", "cmnist_gaussian_120"], ["uniform", "cmnist_gaussian_uniform"]]
dataset_lengths = ["5k"]

tot_accs2 = []
tot_accs3 = []

print(dataset_lengths[0][:-2])
for dl in dataset_lengths:
    for x, y in datasets:
        n_exp = 1
        tot_accs2.append(avg_acc(x, y, 2, dl, n_exp))
        tot_accs3.append(avg_acc(x, y, 3, dl, n_exp))

In [27]:
y2_mean = []
y2_std = []
y3_mean = []
y3_std = []
for i in range(len(tot_accs2)):
    y2_mean.append(np.mean(tot_accs2[i]))
    y2_std.append(np.std(tot_accs2[i]))
    y3_mean.append(np.mean(tot_accs3[i]))
    y3_std.append(np.std(tot_accs3[i]))

In [ ]:
print("y2_mean = " + str(y2_mean))
print("y2_std = " + str(y2_std))
print("y3_mean = " + str(y3_mean))
print("y3_std = " + str(y3_std))

In [29]:
np.savetxt("plots/pt_color_uniform.txt", [y2_mean, y2_std, y3_mean, y3_std])

In [11]:
datasets = [["0", "cmnist_deterministic"], ["12", "cmnist_gaussian_12"], ["60", "cmnist_gaussian_60"],
            ["120", "cmnist_gaussian_120"], ["uniform", "cmnist_gaussian_uniform"]]
datasets = [["0", "color_uniform"], ["12", "color_uniform"], ["60", "color_uniform"],
            ["120", "color_uniform"], ["uniform", "color_uniform"]]
dataset_lengths = ["1k", "2k", "5k", "10k", "20k", "60k"]
dataset_lengths = ["5k"]
n_exp = "best"
model_layers2 = [84, "M", 165, "M", 333, "M"]
model_layers3 = [32, "M", 64, "M", 128, "M"]
features = [[1, 1], [3, 1, 1]]

for dl in dataset_lengths:
    tot_accs2 = []
    tot_accs3 = []
    for x, y in datasets:
        tot_accs2.append(avg_acc(x, y, 2, dl, n_exp, model_layers2, model_layers3, features))
        tot_accs3.append(avg_acc(x, y, 3, dl, n_exp, model_layers2, model_layers3, features))

    y2_mean = []
    y2_std = []
    y3_mean = []
    y3_std = []
    for i in range(len(tot_accs2)):
        y2_mean.append(np.mean(tot_accs2[i]))
        y2_std.append(np.std(tot_accs2[i]))
        y3_mean.append(np.mean(tot_accs3[i]))
        y3_std.append(np.std(tot_accs3[i]))

    np.savetxt("Experiments/cmnist/plots/best/color_uniform.txt", [y2_mean, y2_std, y3_mean, y3_std])

uniform color_uniform 3 5k: 100%|██████████| 6/6 [00:13<00:00,  2.30s/it]
